In [1]:
%pip install pandas
%pip install splink
%pip install usaddress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 627.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 1.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 2.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 1.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 1.1 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 2.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 1.7 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.6/260.6 k

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('../lib'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [12]:
import pandas as pd
import numpy as np
from parse.address_parser import parse_address, OCCUPANCY_TYPES_REVERSED

noisy_persons = pd.read_csv(
    '~/data/noisy_persons.csv', 
    index_col=0, 
    dtype={'unique_id': str, 'firstname': str, 'middlename': str, 'prefix': str, 'suffix': str, 'lastname': str, 'date_of_birth': 'str', 'raw_address': str, 'social_security_number': str, 'sex': str, 'phone': str, 'personal_email': str, 'corporate_email': str}, 
    # parse_dates=['date_of_birth']
)
noisy_persons = noisy_persons.replace({np.nan: None})

# Cleansing
#   1. Address
#     1.1. Parse address
parsed_addresses = noisy_persons['raw_address'].apply(parse_address)
parsed_df = pd.DataFrame(parsed_addresses.tolist())
noisy_persons = pd.concat([noisy_persons.drop(['raw_address'], axis='columns'), parsed_df], axis=1)
#     1.2. Normalize occupancy type and directions
def normalize_abbrv(d: dict):
    def normalize(text):
        lower = text.lower()
        if lower in d:
            return d[lower]
        if lower.endswith('.'):
            lower = lower[:-1]
        return lower
    return normalize
noisy_persons['occupancy_type'] = noisy_persons['occupancy_type'].apply(normalize_abbrv(OCCUPANCY_TYPES_REVERSED))
# TODO: Directional terms & street suffixes

#  2. Normalize all strings
noisy_persons['firstname'] = noisy_persons['firstname'].str.upper()
noisy_persons['middlename'] = noisy_persons['middlename'].str.upper()
noisy_persons['prefix'] = noisy_persons['prefix'].str.upper()
noisy_persons['suffix'] = noisy_persons['suffix'].str.upper()
noisy_persons['lastname'] = noisy_persons['lastname'].str.upper()
noisy_persons['town'] = noisy_persons['town'].str.upper()
noisy_persons['state'] = noisy_persons['state'].str.upper()
noisy_persons['fpo_apo'] = noisy_persons['fpo_apo'].str.upper()
noisy_persons['occupancy_type'] = noisy_persons['occupancy_type'].str.upper()
def normalize_sex(text):
    if text is None:
        return None
    return 'M' if text == 'Sex.MALE' else 'F'
noisy_persons['sex'] = noisy_persons['sex'].apply(normalize_sex)


noisy_persons

,unique_id,firstname,middlename,prefix,suffix,lastname,date_of_birth,social_security_number,sex,phone,...,street_suffix,post_directional,occupancy_type,secondary_number,po_box_type,po_box_id,town,postcode,state,fpo_apo
0,70955c3f-1e76-4aea-b09f-67404ff282e0,LES,None,None,None,BROWN,2001-01-11,None,F,678-529-1324x66540,...,None,None,APT,744,,,PORT JESSICABERG,03651,DC,None
1,a9e1173f-09f1-4659-9bde-3b7bc4038016,LESLIE,None,None,None,BROWN,None,136-55-5752,F,678-529-1342x66540,...,None,None,APT,744,,,PORT JESSICABERG,03751,DC,None
2,5a1f318c-f0c7-460e-a17b-c050afd45bf0,LESLIE,None,None,None,BROWN,1970-01-01,None,F,678-529-1342x66540,...,None,None,APT,744,,,PORT JESSICABERG,03561,DC,None
3,b7644184-17f5-4523-9d84-0aca333bfb51,LESLIE,None,None,None,BROWN,1970-01-01,136-55-5752,F,678-529-1342x66540,...,None,None,APT,744,,,PORT JESSICABERG,03561,DC,None
4,cf0d2f67-d7f6-46b8-b933-1770af9e96a8,LES,None,None,None,BROVNN,1970-01-01,None,F,678-529-1342x65640,...,None,None,,,,,APARTMENT PORT JESSICABERG,03651,DC,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13764,3a6af67b-d050-4e5a-983a-31f65695a9eb,JAMIE,None,None,None,HANNA,1953-01-24,None,F,(574)417-0970x497,...,None,None,STE,18,,,LAKE TERESA,67564,MH,None
13765,9b11517f-d65e-4324-8d36-820af8b1c0b6,JAMIE,None,None,None,HENNE,None,None,None,(574)417-0970x497,...,None,None,STE,318,,,LXKE TERESA,67564,MH,None
13766,192dc27e-34fa-4639-8df3-5320ab480509,JAMIE,None,None,None,HANNA,1953-01-24,506-93-3909,M,(574)417-0970x497,...,None,None,STE,318,,,LAKE TERESA,67564,MH,None
13767,3ef46d3c-f2a8-46ca-921b-1b512373b824,JAMIE,None,None,None,HANNA,None,506-93-3909,F,(574)417-0907x497,...,None,None,STE,318,,,LAKE TERESA,67564,MH,None
